<a href="https://colab.research.google.com/github/Chenyan-Lu/GIS322/blob/main/notebook/Module_5_(2)_Table_Join_and_Spatial_Join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.2 Attribute join and spatial join


<br></br>
There are two ways to combine datasets in `geopandas` – attribute joins and spatial joins.
<br></br>
<br></br>
In an <b>attribute join</b>, a `GeoSeries` or `GeoDataFrame` is combined with a regular pandas Series or DataFrame based on a <b>common variable</b>. This is analogous to normal merging or joining in pandas. An example would be joining student grade to student information based on student ID as the <b>key</b>.
<br></br>
<br></br>
In a <b>spatial join</b>, observations from to GeoSeries or GeoDataFrames are combined based on their <b> spatial relationship</b> to one another. An example would be finding in which zipcode a crime incident happend by looking at which zipcode polygon contains that crime incident location. This operation definately needs more computation, but all is under the hood.

<br></br>
<br></br>
In the below example, we will be using sample datasets from `geopandas` library to demostrate. Note that there is one extra line of code to install a `rtree` package. You <b>have to do this</b> in order to make spatial join works in Google Colab.


In [1]:
!pip install geopandas
#Sorry this is an additional line of code you need in Google Colab in order to make things work
!sudo apt install python3-rtree
import geopandas
%matplotlib inline

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6 python3-rtree
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 365 kB of archives.
After this operation, 1,799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-c6 amd64 1.9.3-2 [55.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-dev amd64 1.9.3-2 [16.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-rtree all 0.9.7-1 [46.4 kB]
Fetched 365 kB in 1s (395 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog

## Attribute Joins
<br></br>
Attribute joins are accomplished using the merge method. In general, it is recommended to use the merge method called from the spatial dataset. With that said, the stand-alone merge function will work if the GeoDataFrame is in the left argument; if a DataFrame is in the left argument and a GeoDataFrame is in the right position, the result will no longer be a GeoDataFrame.
<br></br>
<br></br>






For example, consider the following merge that adds full names to a GeoDataFrame that initially has only ISO codes for each country by merging it with a pandas DataFrame.


In [2]:
world = geopandas.read_file('/content/ne_110m_admin_0_sovereignty.zip')

country_shapes = world[['geometry', 'ISO_A3']]
country_names = world[['NAME', 'ISO_A3']]

The first table `country_shapes` contains geometry and iso code of a given country.

In [3]:
country_shapes.head()

,geometry,ISO_A3
0,"MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ...",FJI
1,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...",TZA
2,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",ESH
3,"MULTIPOLYGON (((-122.84 49, -122.97421 49.0025...",CAN
4,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312...",-99


The second table `country_names` contains name and iso code of a given country.

In [4]:
country_names.head()

,NAME,ISO_A3
0,Fiji,FJI
1,Tanzania,TZA
2,W. Sahara,ESH
3,Canada,CAN
4,United States of America,-99


Lets merge two tables `country_names` with `country_shapes` using `merge` method on shared variable (iso codes `iso_a3`). The resulting table would have both name and geometry in one single table.

In [5]:
country_merged = country_shapes.merge(country_names, on='ISO_A3')
country_merged.head()

,geometry,ISO_A3,NAME
0,"MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ...",FJI,Fiji
1,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...",TZA,Tanzania
2,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",ESH,W. Sahara
3,"MULTIPOLYGON (((-122.84 49, -122.97421 49.0025...",CAN,Canada
4,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312...",-99,United States of America


## Spatial Joins
<br></br>
Spatial joins are joining based on geometries. In this example, we have a list of cities that we know their coordinates, but we don't know which country each city is within. And spatial join can help in this case. The command in `geopandas` is `sjoin`.
<br></br>
<br></br>






In [6]:
# download ne_110m_populated_places.zip from link: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
cities = geopandas.read_file('/content/ne_110m_populated_places.zip')

In [7]:
cities.head()

,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,NAMEASCII,ADM0CAP,CAPIN,...,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,8,10,3,Admin-0 capital,Vatican City,None,None,Vatican City,1,None,...,None,None,None,None,None,None,None,None,None,POINT (12.45339 41.90328)
1,7,20,0,Admin-0 capital,San Marino,None,None,San Marino,1,None,...,None,None,None,None,None,None,None,None,None,POINT (12.44177 43.9361)
2,7,20,0,Admin-0 capital,Vaduz,None,None,Vaduz,1,None,...,None,None,None,None,None,None,None,None,None,POINT (9.51667 47.13372)
3,6,30,8,Admin-0 capital alt,Lobamba,None,None,Lobamba,0,Legislative and,...,None,None,None,None,None,None,None,None,None,POINT (31.2 -26.46667)
4,6,30,8,Admin-0 capital,Luxembourg,None,None,Luxembourg,1,None,...,None,None,None,None,None,None,None,None,None,POINT (6.13 49.61166)


<br></br>
Let's play around with those methods using the world countries example.</font>

In [8]:
countries = world[['geometry', 'NAME']]
countries = countries.rename(columns={'name':'country'})

In [9]:
countries.head()

,geometry,NAME
0,"MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ...",Fiji
1,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...",Tanzania
2,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",W. Sahara
3,"MULTIPOLYGON (((-122.84 49, -122.97421 49.0025...",Canada
4,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312...",United States of America


<br></br>
Let's execute the spatila join with the `sjoin()` command.</font>

In [10]:
# Execute spatial join
cities_with_country = geopandas.sjoin(cities, countries, how="inner", predicate='intersects')
cities_with_country.head()

,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME_left,NAMEPAR,NAMEALT,NAMEASCII,ADM0CAP,CAPIN,...,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry,index_right,NAME_right
0,8,10,3,Admin-0 capital,Vatican City,None,None,Vatican City,1,None,...,None,None,None,None,None,None,None,POINT (12.45339 41.90328),137,Italy
1,7,20,0,Admin-0 capital,San Marino,None,None,San Marino,1,None,...,None,None,None,None,None,None,None,POINT (12.44177 43.9361),137,Italy
2,7,20,0,Admin-0 capital,Vaduz,None,None,Vaduz,1,None,...,None,None,None,None,None,None,None,POINT (9.51667 47.13372),111,Austria
3,6,30,8,Admin-0 capital alt,Lobamba,None,None,Lobamba,0,Legislative and,...,None,None,None,None,None,None,None,POINT (31.2 -26.46667),71,eSwatini
4,6,30,8,Admin-0 capital,Luxembourg,None,None,Luxembourg,1,None,...,None,None,None,None,None,None,None,POINT (6.13 49.61166),125,Luxembourg


<br></br>
And here we go, we can see that each city is now joined with the country. For example, Vatican City is joined with Italy.

## join function Parameters
<br></br>
 If you notice, there are two arguments in the `sjoin()` function above: `how="inner"` and `op='intersects'`.

### predicate
<br></br>
The `predicate` argument specifies how GeoPandas decides whether or not to join the attributes of one object to another, based on their geometric relationship.

The values for `predicate` correspond to the names of geometric binary predicates and depend on the spatial index implementation.

The default spatial index in GeoPandas currently supports the following values for `predicate` which are defined in the Shapely documentation:

* *intersects*

* *contains*

* *within*

* *touches*

* *crosses*

* *overlaps*

You can read more about each join type in the [Shapely documentation](https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates).

<br></br>
### how

The `how` argument specifies the type of join that will occur and which geometry is retained in the resultant geodataframe. It accepts the following options:

* `left`: use the index from the first (or *left_df*) geodataframe that you provide to sjoin; retain only the left_df geometry column
* `right`: use index from second (or *right_df*); retain only the *right_df* geometry column
* `inner`: use intersection of index values from both geodataframes; retain only the *left_df* geometry column


Note more complicated spatial relationships can be studied by combining geometric operations with spatial join. To find all polygons within a given distance of a point, for example, one can first use the `buffer()` method to expand each point into a circle of appropriate radius, then intersect those buffered circles with the polygons in question.